In [ ]:
import csv
import random
import string
import os
import requests
from pymongo import MongoClient

In [ ]:
mongo_uri = 'your_mongo_uri'
file_path = "/Users/harrypapadakis/Desktop/stress_tests/"

In [ ]:
def bulk_insert(file_path, endpoint, unique_identifier='username'):
    """
    General function to bulk insert data from a CSV file to a microservice endpoint.

    :param file_path: Path to the CSV file.
    :param endpoint: The endpoint of the microservice.
    :param unique_identifier: A field to uniquely identify records (for logging purposes).
    """
    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            response = requests.post(endpoint, json=row)
            if response.status_code != 200:
                print(f"Failed to insert {unique_identifier}: {row[unique_identifier]}, Status code: {response.status_code}")


In [ ]:
def retrieve_from_mongo(mongo_uri, db_name, collection_name, filter={}, projection=None):
    """
    General function to retrieve data from a MongoDB collection.

    :param mongo_uri: URI of the MongoDB server.
    :param db_name: Name of the database.
    :param collection_name: Name of the collection.
    :param filter: A dictionary specifying the query filter.
    :param projection: A dictionary specifying the fields to include or exclude.
    :return: List of documents retrieved from the collection.
    """
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]
    documents = collection.find(filter, projection)
    return list(documents)

In [ ]:
def generate_username():
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=12))

def generate_email(username):
    domains = ['example.com', 'test.com', 'sample.org']
    return f"{username}@{random.choice(domains)}"

def generate_password():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=10))

filename_users = 'users.csv'
filename_login = 'login.csv'
num_records = 200

# Create users.csv
with open(file_path + filename_users, mode='w', newline='') as file_users:
    writer_users = csv.writer(file_users)
    writer_users.writerow(['username', 'email', 'password', 'repassword', 'isAdmin'])

    # Create login.csv
    with open(file_path + filename_login, mode='w', newline='') as file_login:
        writer_login = csv.writer(file_login)
        writer_login.writerow(['email', 'password'])

        for _ in range(num_records):
            username = generate_username()
            email = generate_email(username)
            password = generate_password()
            repassword = password  # Assuming repassword is the same as password
            isAdmin = 0
            writer_users.writerow([username, email, password, repassword, isAdmin])
            writer_login.writerow([email, password])

print(f"CSV files '{file_path + filename_users}' and '{file_path + filename_login}' with {num_records} records each created successfully.")


In [ ]:
user_service_endpoint = 'http://localhost:3005/signup'
file_path_users = "/Users/harrypapadakis/Desktop/stress_tests/users.csv"
bulk_insert(file_path_users, user_service_endpoint, unique_identifier='email')

In [ ]:
db_name = 'usersDB'
collection_name = 'Users'
users = retrieve_from_mongo(mongo_uri, db_name, collection_name)
print(users)

In [ ]:
# Function to generate a random string
def generate_random_string(length=8):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

# Function to create a new CSV for submissions with specific parameters
def create_submission_csv(file_path, users):
    fieldnames = ['name', 'username', 'userId', 'numVehicles', 'depot', 'maxDistance', 'pythonFilePath', 'jsonFilePath']
    with open(file_path + "submissions.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)

        problem_specs = [
            {'numVehicles': 10, 'jsonFilePath': 'locations_1000.json'},  # 1st problem: 1000 locations, 10 vehicles
            {'numVehicles': 50, 'jsonFilePath': 'locations_200.json'},   # 2nd problem: 200 locations, 50 vehicles
        ]

        # Next 120 problems: 20 locations, 5 vehicles
        for _ in range(120):
            problem_specs.append({'numVehicles': 5, 'jsonFilePath': 'locations_20.json'})
    
        # Next 113 problems: 200 locations, 50 vehicles
        for _ in range(113):
            problem_specs.append({'numVehicles': 50, 'jsonFilePath': 'locations_200.json'})
        
        # Next 60 problems: 20 locations, 5 vehicles
        for _ in range(60):
            problem_specs.append({'numVehicles': 5, 'jsonFilePath': 'locations_20.json'})
        
        # Next 5 problems: 200 locations, 50 vehicles
        for _ in range(5):
            problem_specs.append({'numVehicles': 50, 'jsonFilePath': 'locations_200.json'})

        for spec in problem_specs:
            user = random.choice(users)
            record = {
                'name': generate_random_string(10),
                'username': user['username'],
                'userId': str(user['_id']),
                'numVehicles': spec['numVehicles'],
                'depot': 0,
                'maxDistance': 30000,
                'pythonFilePath': file_path + "vrpSolver.py",  # Specify the actual path to the Python solver
                'jsonFilePath': file_path + spec['jsonFilePath']  # Specify the actual path to the JSON file
            }
            writer.writerow([record[field] for field in fieldnames])
    
    print(f"""CSV file '{file_path + "submissions.csv"}' created successfully.""")


# Create new CSV with submission data
create_submission_csv(file_path, users)

In [ ]:
# Function to generate a random string
def generate_random_string(length=8):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

# Function to create a new CSV for submissions with specific parameters
def create_submission_csv(file_path, users):
    fieldnames = ['name', 'username', 'userId', 'numVehicles', 'depot', 'maxDistance', 'pythonFilePath', 'jsonFilePath']
    with open(file_path + "submissions.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)

        problem_specs = [
        ]

        for _ in range(1000):
            problem_specs.append({'numVehicles': 5, 'jsonFilePath': 'locations_20.json'})

        for spec in problem_specs:
            user = random.choice(users)
            record = {
                'name': generate_random_string(10),
                'username': user['username'],
                'userId': str(user['_id']),
                'numVehicles': spec['numVehicles'],
                'depot': 0,
                'maxDistance': 30000,
                'pythonFilePath': file_path + "vrpSolver.py",  # Specify the actual path to the Python solver
                'jsonFilePath': file_path + spec['jsonFilePath']  # Specify the actual path to the JSON file
            }
            writer.writerow([record[field] for field in fieldnames])
    
    print(f"""CSV file '{file_path + "submissions.csv"}' created successfully.""")


# Create new CSV with submission data
create_submission_csv(file_path, users)

In [ ]:
import csv
import requests

def bulk_insert_submissions(file_path, endpoint, python_file_field='pythonFile', json_file_field='jsonFile'):
    """
    General function to bulk insert data from a CSV file to a microservice endpoint for creating submissions.

    :param file_path: Path to the CSV file.
    :param endpoint: The endpoint of the microservice.
    :param python_file_field: Field name for the Python file in the request.
    :param json_file_field: Field name for the JSON file in the request.
    """
    with open(file_path + "submissions.csv", mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            files = {
                python_file_field: open(row['pythonFilePath'], 'rb'),
                json_file_field: open(row['jsonFilePath'], 'rb')
            }
            data = {
                'name': row['name'],
                'username': row['username'],
                'userId': row['userId'],
                'numVehicles': row['numVehicles'],
                'depot': row['depot'],
                'maxDistance': row['maxDistance']
            }
            response = requests.post(endpoint, files=files, data=data)
            if response.status_code != 201:
                print(f"Failed to insert submission for user: {row['username']}, Status code: {response.status_code}")
            for f in files.values():
                f.close()

# Example usage:
file_path_submissions = file_path + "submissions.csv"
create_submission_service_endpoint = 'http://localhost:3001/submission/create'

bulk_insert_submissions(file_path, create_submission_service_endpoint)
print("Bulk insert for submissions completed.")

In [ ]:
db_name = 'problemsDB'
collection_name = 'problems'
problems = retrieve_from_mongo(mongo_uri, db_name, collection_name)
print(problems)

In [ ]:
def create_problem_ids_csv(problems, file_path):
    fieldnames = ['problemId']
    with open(file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for problem in problems:
            problem_row = {
                'problemId': str(problem['submissionId'])
            }
            writer.writerow(problem_row)

file_path_problems = file_path + 'problems.csv'
create_problem_ids_csv(problems, file_path_problems)